In [2]:
import numpy as np
import pandas as pd 
import tensorflow as tf 
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import BatchNormalization, Dense, Embedding, Input, Concatenate, Flatten, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import one_hot
from tensorflow.keras.utils import plot_model

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,Unnamed: 0,latitude,longitude,company,is_local,type,fin_1,fin_2,fin_3,fin_4,target
0,0,40.10891,-83.09286,8336,0,3,-135060.089443,86013.396489,1206.094242,52287.082257,0
1,1,39.86542,-84.06280,18403,1,0,-1766.845055,14985.640180,477.494992,168836.215743,1
2,2,39.10266,-84.52468,14022,0,3,-177302.873693,44881.958005,1463.339889,130388.243325,0
3,3,39.10148,-84.52341,11051,0,0,209049.997460,0.000000,95.340075,103267.727546,1
4,4,41.06213,-81.53784,3243,0,3,8669.269507,0.000000,399.421926,177532.206618,1


In [4]:
def transform_df(df):
    df = df.drop(columns=df.columns[0])
    mask = df['company'].value_counts()
    df['company'] = np.where(df['company'].isin(mask.index[mask>=10]), df.company, -1)
    df['geo'] = KMeans(n_clusters=300, random_state=42).fit_predict(df[['latitude', 'longitude']])
    df = df.drop(columns=['latitude', 'longitude'])
    return df

In [5]:
%%time
train_df = transform_df(train_df)
test_df = transform_df(test_df)

Wall time: 1min 21s


In [6]:
le = LabelEncoder()
le.fit(train_df.company)
train_df['company'] = le.transform(train_df.company)

In [7]:
test_df['company'] = np.where(test_df['company'].isin(train_df['company'].unique()), test_df['company'], -1)
test_df['company'] = le.transform(test_df.company)

In [8]:
train_df.company.max()

1483

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(train_df.drop('target', axis=1), 
                                                      train_df.target, test_size=0.1, random_state=42)

In [10]:
numeric_cols = ['fin_1', 'fin_2', 'fin_3', 'fin_4', 'is_local']

In [30]:
def build_model(numeric_data, company_data, type_data, geo_data, num_company):
    numeric_input = Input(shape=(5,), dtype=tf.float32, name="numeric_data")
    numeric = Dense(32, activation='relu')(numeric_input)
    numeric = BatchNormalization()(numeric)
    numeric = Dense(32, activation='relu')(numeric)
    numeric = BatchNormalization()(numeric)
    numeric = Reshape((1, 32))(numeric)

    
    company_input = Input(shape=(1,), dtype=tf.float32, name="company_data")
    company = Embedding(num_company + 1, 64)(company_input)
    company = Dense(32, activation='relu')(company)
    # company = Reshape((-1, 32))(company)

    type_l_input = Input(shape=(1,), dtype=tf.float32, name="type_data")
    type_l = Embedding(6, 8)(type_l_input)
    # type_l = Reshape((-1, 8))(type_l)

    geo_input = Input(shape=(1,), dtype=tf.float32, name="geo_data")
    geo = Embedding(300, 32)(geo_input)
    geo = Dense(32, activation='relu')(geo)
    # geo = Reshape((-1, 32))(geo)

    
    out = Concatenate()([numeric, company, type_l, geo])
    out = Dense(64, activation='relu')(out)
    out = BatchNormalization()(out)
    out = Dense(64, activation='relu')(out)
    out = BatchNormalization()(out)
    out = Dense(1, activation="sigmoid")(out)
    
    model = Model(inputs=[numeric_input, company_input, type_l_input, geo_input], outputs=out)
    model.compile(Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

model = build_model(train_df[numeric_cols].values, train_df.company.values, 
                    train_df.type.values, train_df.geo.values, train_df.company.max())

In [37]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
numeric_data (InputLayer)       [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 32)           192         numeric_data[0][0]               
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 32)           128         dense_7[0][0]                    
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 32)           1056        batch_normalization_4[0][0]      
____________________________________________________________________________________________

In [40]:
plot_model(model, to_file='model.png')

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [32]:
BATCH_SIZE = 128

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices(({"numeric_data": X_train[numeric_cols].values, 
                          "company_data": X_train.company.values, 
                          "type_data": X_train.type.values, 
                          "geo_data": X_train.geo.values}, y_train))
    .shuffle(2048)
    .batch(BATCH_SIZE)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices(({"numeric_data": X_valid[numeric_cols].values, 
                          "company_data": X_valid.company.values, 
                          "type_data": X_valid.type.values, 
                          "geo_data": X_valid.geo.values}, y_valid))
    .batch(BATCH_SIZE)

)

In [33]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='.',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [34]:
%%time
n_steps = X_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=10, 
    callbacks=[model_checkpoint_callback]
)

Epoch 1/10
1125/1125 [==============================] - 6s 3ms/step - loss: 0.5962 - accuracy: 0.6895 - val_loss: 0.4975 - val_accuracy: 0.7671
Epoch 2/10
1125/1125 [==============================] - 4s 3ms/step - loss: 0.4956 - accuracy: 0.7624 - val_loss: 0.4710 - val_accuracy: 0.7716
Epoch 3/10
1125/1125 [==============================] - 3s 3ms/step - loss: 0.4775 - accuracy: 0.7661 - val_loss: 0.4590 - val_accuracy: 0.7749
Epoch 4/10
1125/1125 [==============================] - 4s 3ms/step - loss: 0.4672 - accuracy: 0.7670 - val_loss: 0.4514 - val_accuracy: 0.7742
Epoch 5/10
1125/1125 [==============================] - 3s 3ms/step - loss: 0.4610 - accuracy: 0.7695 - val_loss: 0.4467 - val_accuracy: 0.7745
Epoch 6/10
1125/1125 [==============================] - 3s 3ms/step - loss: 0.4563 - accuracy: 0.7696 - val_loss: 0.4435 - val_accuracy: 0.7748
Epoch 7/10
1125/1125 [==============================] - 3s 3ms/step - loss: 0.4534 - accuracy: 0.7697 - val_loss: 0.4409 - val_accuracy:

In [35]:
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(({"numeric_data": test_df[numeric_cols].values, 
                          "company_data": test_df.company.values, 
                          "type_data": test_df.type.values, 
                          "geo_data": test_df.geo.values}))
    .batch(BATCH_SIZE)
)
predictions = model.predict(test_dataset, verbose=1).argmax(axis=1)

313/313 [==============================] - 1s 2ms/step


In [36]:
accuracy_score(test_df.target.values, predictions.reshape(-1))

0.40665